In [1]:
from arcgis.gis import GIS
from arcgis import features as fs
import glob
import os
import pandas as pd

jgeo_path = '/home/greg/data/rawdata/JornadaGeospatial'

In [2]:
# Some stuff we need to access the NMSU ArcGIS Online system
import sys
sys.path.append('/home/greg/GitHub/jgeo_utils')
sys.path.append('/home/greg/admin/')
import jgeo_py.agol as agol
import jgeo_py.pg as pg
import nmsu_agol_cred # AGOL credentials
agol_url=nmsu_agol_cred.agol_url
agol_user=nmsu_agol_cred.agol_user
password=nmsu_agol_cred.password
jornadagis_grpid=nmsu_agol_cred.jornadagis_grpid
john_ragosta_grpid = nmsu_agol_cred.john_ragosta_grpid

# Log into NMSU AGOL and get a GIS object
gis = GIS(agol_url, username=agol_user,
    password=password)

# Groups


# Metadata file
metadata = pd.read_csv(os.path.join(jgeo_path, 'metadata', 'JornadaGeosp_metadata_20220107.csv'))

In [40]:
# Get groupids
jgrp_id = gis.groups.search('title:JornadaGIS')[0].id
jgrp_pub_id = gis.groups.search('title:Jornada GIS Portal')[0].id

In [16]:
my_content = gis.content.search(query="owner:" + gis.users.me.username, 
                                item_type="folder", 
                                max_items=5)
my_content

[]

In [5]:
metadata.head()

,package_name,gpkg_fname,layer_name,geom_type,attributes,n_features,origin_gdb,package_abstract,package_tags,layer_abstract,layer_summary,layer_tags,metadata_link,data_link,contact_name,contact_email,license,access_use_text,date_read
0,JornadaVegetation,JornadaVegetation.gpkg,CDRRCGrassCondition1858,MultiPolygon,"['OBJECTID', 'PERIMETER', 'CRGRASS_18', 'CRGRA...",13,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:35.836133
1,JornadaVegetation,JornadaVegetation.gpkg,CDRRCShrubPresence1858,MultiPolygon,"['OBJECTID', 'AREA_', 'PERIMETER', 'CRSHRUB_18...",29,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:35.996706
2,JornadaVegetation,JornadaVegetation.gpkg,JERGrassCondition1858,MultiPolygon,"['OBJECTID', 'PERIMETER', 'JERGRASS_1', 'JERGR...",20,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:36.156854
3,JornadaVegetation,JornadaVegetation.gpkg,JERShrubPresence1858,MultiPolygon,"['OBJECTID', 'GIS_IMS_jer_shrub_presence_18_2'...",32,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:36.314905
4,JornadaVegetation,JornadaVegetation.gpkg,JERVegetation1915,MultiPolygon,"['OBJECTID', 'AREA_', 'PERIMETER', 'SVEG12_', ...",218,All_studies_JornadaGDB_03_30_21.gdb,This layer is part of a collection of Jornada ...,"Jornada Basin, JornadaVegetation, vegetation, ...",layer abstract,layer summary,layer tags,metadata link,data link,Jornada IM Team,jornada.data@nmsu.edu,Creative Commons Attribution 4.0,Restricted to users who have completed the Jor...,2022-01-07 10:31:36.476252


## Copy research project geopackages to AGOL

In [50]:
import importlib as imp
imp.reload(ag)
import pdb

# Get a sorted list of all the project geopackages
prj_gpkgs = sorted(glob.glob(os.path.join(jgeo_path, "jrn_studies", "/*.gpkg")))

# Create a folder for GeoJSON items and for the 
# packages (FeatureLayerCollections)
folder_geojson = 'JornadaResearchProjects_geojson'
gis.content.create_folder(folder_geojson)
folder_pkg = 'JornadaResearchProjects'
gis.content.create_folder(folder_pkg)

for i, pkg_path in enumerate(prj_gpkgs[0:6]):
    pkg_name = pkg_path.split('/')[1][0:6]
    studyid = pkg_name[3:6]
    print(pkg_name)
    # Get the metadata
    pkg_md = metadata[metadata.gpkg_fname==pkg_name+'.gpkg']
    
    # Create a hosted feature service on AGOL
    new_svc = gis.content.create_service(name=pkg_name,
        service_type='featureService', folder=folder_pkg)
    # Create a featureLayerCollection with new_svc
    new_flc = fs.FeatureLayerCollection.fromitem(new_svc)
    # Share the feature service with the JornadaGIS group
    new_svc.share(everyone=False, org=False, groups=[jornadagis_grpid])

    # Update the properties of the new featureLayerCollection (title,
    # summary, description, etc)
    fs_defs = ag.jornada_fs_definitions(new_flc, pkg_md)
    # A little kludge to fix the geopackage description
    fs_defs['description'] = fs_defs['description'].replace(
        ' layer is part of ', ' is ')
    new_flc.manager.update_definition(fs_defs)
    new_svc.update(fs_defs) # This updates the item info?
    # Copy the geopackage layers into the featureLayerCollection
    ag.geopkg_to_multi_fs(gis, folder_geojson, new_flc, pkg_path,
        pkg_md, clean_local=True)

    

prj001
True
True
True
True
True
prj002
True
prj004
True
prj008
True
prj009
True
True
prj010
True
True


In [49]:
folder_geojson = 'JornadaResearchProjects_geojson'
folder_pkg = 'JornadaResearchProjects'
gis.content.delete_folder(folder_geojson)
gis.content.delete_folder(folder_pkg)

Folder doesn't exist.
Folder doesn't exist.


False

## Copy other geopackages

In [7]:
import importlib as imp
imp.reload(ag)

gpkgs = ['JornadaVegetation.gpkg',
    'JornadaBounds&Structures.gpkg',
    'JornadaMonitoringStations.gpkg',
    'JornadaLandforms&Soil.gpkg',
    'JornadaMisc.gpkg']

for i, pkg_path in enumerate(gpkgs[0:5]):
    # Create a folder
    folder_pkg = pkg_path.split('.')[0]
    gis.content.create_folder(folder_pkg)

    # Get the metadata
    pkg_md = metadata[metadata.package_name==folder_pkg]
    
    # Copy the geopackage layers into the featureLayerCollection
    items, fslist = ag.geopkg_to_single_fs(gis, folder_pkg, pkg_path,
        pkg_md, clean_local=True)

    # Share the feature layers with groups
    for i in range(len(fslist)):
        fslist[i].share(everyone=False, org=False,
            groups=[jornadagis_grpid, johntest_grpid])

CDRRCGrassCondition1858
CDRRCShrubPresence1858
JERGrassCondition1858
JERShrubPresence1858
JERVegetation1915
JERVegetation1928
JERVegetation1998
Buffington_Herbel_1858
Buffington_Herbel_1915
Buffington_Herbel_1928
Buffington_Herbel_1963
1938_CDRRC_Vegetation
1998_CDRRC_Vegetation
Boundary_JER_full
Boundary_CDRRC_JER
All3_Bdry_25Aug15
Airstrip
BurnPlots21Aug13
Cattleguards
CDNP_Bdry_25Aug15
CDRRC_Bdry_25Aug15
CDRRC_ExcloEDITED21Aug13
CDRRC_Exclosures_17Sep15
ClosedRoad
CSISParking
DefunctTowers
Dirt_Tanks
ExclosuresJER_17Jul2015
ExclusuresJER_Jul172015
JER_Bdry_25Aug15
JER_CDRRC_Bdry_FromCorrectedMar2014
JER_DeededBndry_OCt2013
JER_FtB_WSMR_bndry
Lvstk_Corral_Lot_Lane
Pastures_CDRRC2_16Sep15
Pastures_JER_16Sep15
RanchStructures
random_troughs
skipping layer: random_troughs
Rds_ClipToJER_27Apr18
RemvdFenceLabel
skipping layer: RemvdFenceLabel
RemvdFences
Tanks
Towers3
TwdieTower_WGS84
UTEP_Site_Access_Trail_UTM
Wells
AllRds_JER_And_Vicinity_04Sep15
Fence_JER_06Nov15
Fence_CDRRC_26Aug15
Jo

In [4]:
folder_geojson = 'JornadaMonitoringStations'
#folder_pkg = 'JornadaResearchProjects'
#gis.content.delete_folder(folder_geojson)
#gis.content.delete_folder(folder_pkg)

TypeError: 'NoneType' object is not subscriptable

In [8]:
items[1]

<Item title:"cr_rg_point" type:GeoJson owner:gmaurer1>

In [9]:
fslist[1]

<Item title:"cr_rg_point" type:Feature Layer Collection owner:gmaurer1>

In [6]:
from arcgis.gis import GIS
import sqlalchemy as sqla
import geopandas as gpd
import pandas as pd
import os
# Get credentials for the jgeo database
import sys
sys.path.append('/home/greg/admin/')
import jgeo_cred as dbcred
# Some stuff we need to access the NMSU ArcGIS Online system
import nmsu_agol_cred # AGOL credentials

# jgeo_utils imports
sys.path.append('/home/greg/GitHub/jgeo_utils')
import jgeo_py.agol as agol
import jgeo_py.pg as pg

# get a connection to jgeo
conn = sqla.create_engine("postgresql+psycopg2://{0}:{1}@{2}:5432/{3}".format(dbcred.user, dbcred.pwd, dbcred.host, dbcred.db))

# Log into NMSU AGOL and get a GIS object
gis = GIS(nmsu_agol_cred.agol_url, 
          username=nmsu_agol_cred.agol_user,
          password=nmsu_agol_cred.password)

In [136]:
# Create our content folders if they don't exist
gis.content.create_folder('jgeo_layers')
gis.content.create_folder('jgeo_layers_public')

{'username': 'gmaurer1',
 'id': 'df57116977dc44748f72bf7fc45aa7b2',
 'title': 'jgeo_layers_public'}

In [143]:
# Inventory of layers
jgeo_online = pd.read_csv(os.path.join(jgeo_path, 'jgeo_layers_online.csv'))
# Get layers that are in jgeo only
jgeo_online = jgeo_online[~pd.isna(jgeo_online.jgeo_table_name)]

In [150]:
import imp
imp.reload(agol)
imp.reload(pg)

for i in jgeo_online[60:].index:
    print(i)
    # Get jgeo layer info from table
    schema = jgeo_online.jgeo_schema[i]
    layer = jgeo_online.jgeo_table_name[i]
    print(layer)
    # Pull metadata from jgeo
    md = pg.md_dict(conn, schema, layer)
    fl_properties = agol.qmd_to_agol(jgeo_online.agol_title[i], md['qmd'])
    # Write to geojson file in temp dir
    geojson_fname = pg.table_to_geojson(conn, schema, layer, 
                                        outdir=os.path.join(jgeo_path, 'tmp'))
    # Publish the layer to jgeo_layers_public if requested
    if jgeo_online.make_public[i]:
        item, fl = agol.add_geojson_item(geojson_fname, gis,
                                         'jgeo_layers_public',
                                         properties=fl_properties,
                                         publish_after=True,
                                         remove_after=True,
                                        )
        # Delete the geojson file from agol
        item.delete()
        # Share with public
        fl.share(everyone=True, org=True,
            groups=[jgrp_id, jgrp_pub_id])
        
    # Or to jgeo_layers if for internal use
    else:
        item, fl = agol.add_geojson_item(geojson_fname, gis, 
                                         'jgeo_layers',
                                         properties=fl_properties,
                                         publish_after=True,
                                         remove_after=True)
        # Delete the geojson file from agol
        item.delete()
        fl.share(everyone=False, org=False,
            groups=[jgrp_id])
    
    #layer = 'prj404_cdrrcvegetation1938_new'
    #test = pg.table_to_gdf(conn, schema, layer)
    #md = pg.md_dict(conn, schema, layer)
#md = pd.read_sql("SELECT * FROM public.qgis_layer_metadata WHERE f_table_name = 'prj404_cdrrcvegetation1938_new';", conn)

72
prj405_jrnphysiography2000
73
prj405_jrnlandforms2000
74
prj405_jrnsoilparentmaterials2000
75
prj420_playas_30
76
prj420_watersheds_30
77
riogrande
78
isaacklakepoly


In [94]:
md['abstract']

'This is a placeholder abstract for the cdrrcgrasscondition1858 geospatial layer (jgeo table name: prj404_cdrrcgrasscondition1858). This layer is part of the Jornada studies collection (jgeo/jrn_studies), which is a collection of geospatial layers describing research locations associated with the USDA-ARS Jornada Experimental Range and Jornada Basin LTER programs. This metadata was automatically generated from a template.'

In [104]:
import xml.etree.ElementTree as ET
root = ET.fromstring(md['qmd'])
# Make some changes to the XML element contents
kw = root.find('keywords')

'long-term experiment,experimental study,field experiment,desert,arid land,environmental monitoring,ecology,ecosystem research'

In [95]:
props={'title':'testing1', 'topics':'Biota,Environment', 'tags':'testtag1,testtag2'}
agol.add_geojson_item(geojson_fname, gis, 'jgeo_layers', properties=props)

<Item title:"testing1" type:GeoJson owner:gmaurer1>

In [135]:
gis.content.delete_folder('jgeo_layers_public')
gis.content.delete_folder('jgeo_layers')

True

In [42]:
search_result = gis.content.search(query="title:1998_CDRRC_Vegetation", item_type="Feature Layer")
search_result[0].id

'c8ba468a64cb4d02a2254c1f82cb0a64'

In [41]:
search_result[0].download_metadata()

'/tmp/metadata.xml'

In [15]:
# This doesn't really work with geometry columns. If there were
# X & Y columns it might
test_fc = gis.content.import_data(test)
test_fc

/home/greg/data/miniconda3/envs/jgeo/lib/python3.8/site-packages/arcgis/features/geo/_io/fileops.py:1027: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  geom_type = df.loc[idx][geom_field].type


KeyError: 'MultiPolygon'

In [16]:
import imp
imp.reload(ag)
item, item_fs = ag.gdf_to_single_fs(gis, 'jrn_website_layers', test, md,
                                    local_folder='/home/greg/data/rawdata/JornadaGeospatial/tmp/',
                                    clean_local=False)

skipping layer: prj404_cdrrcvegetation1938_new
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/greg/data/miniconda3/envs/jgeo/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_582419/483754761.py", line 3, in <module>
    item, item_fs = ag.gdf_to_single_fs(gis, 'jrn_website_layers', test, md,
  File "/home/greg/GitHub/jgeo_utils/jgeo_py/agol.py", line 297, in gdf_to_single_fs
    return(item, item_fs)
UnboundLocalError: local variable 'item' referenced before assignment

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/greg/data/miniconda3/envs/jgeo/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/greg/data/miniconda3/envs/jgeo/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structur